In [ ]:
!pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.2 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=91d2cb0149e2ed817be08c8f465239d0da6679a29cc263693477c70058b9e0fb
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive


In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]



In [ ]:
import random
from sklearn.model_selection import train_test_split

def read_data(file_path):
    sentences = []
    sent = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip() == "":
                if sent:
                    sentences.append(sent)
                    sent = []
            else:
                splits = line.strip().split()
                sent.append((splits[0], splits[-1]))
        if sent:
            sentences.append(sent)
    return sentences

all_sents = read_data(base_path+"dataset/cleaned/NER/converted.bio")

train_sents, test_sents = train_test_split(all_sents, test_size=0.2, random_state=42)

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)


y_pred = crf.predict(X_test)
labels = list(crf.classes_)
labels.remove("O")

print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

print(classification_report(y_test, y_pred, digits=3))
print("F1:", round(f1_score(y_test, y_pred), 4))
print("Precision:", round(precision_score(y_test, y_pred), 4))
print("Recall:", round(recall_score(y_test, y_pred), 4))


                precision    recall  f1-score   support

     B-PATIENT      0.735     0.424     0.538        59
    B-HPO_TERM      0.663     0.469     0.549       512
    I-HPO_TERM      0.636     0.470     0.541       931
B-AGE_FOLLOWUP      0.778     0.318     0.452        22
   B-AGE_ONSET      0.455     0.263     0.333        19
        B-GENE      0.659     0.574     0.614        47
B-GENE_VARIANT      0.778     0.636     0.700        77
I-GENE_VARIANT      0.806     0.765     0.785        98
        I-GENE      0.000     0.000     0.000         2
     I-PATIENT      0.600     0.167     0.261        18
   I-AGE_ONSET      0.571     0.129     0.211        31
I-AGE_FOLLOWUP      0.000     0.000     0.000        22
   B-AGE_DEATH      1.000     0.273     0.429        11
   I-AGE_DEATH      0.800     0.500     0.615         8

     micro avg      0.664     0.474     0.553      1857
     macro avg      0.606     0.356     0.430      1857
  weighted avg      0.655     0.474     0.546 